# Pollaczek Khinchine Formula Simulation for M/G/1

In [29]:
import numpy as np
import random as random
import scipy.stats as stats

## Pollaczek Khinchine Formula
The classic performance of an M/G/1 queue can be obtained by PK Formula.
$$E\left\lbrack R\right\rbrack=\frac{1}{T}\sum_{i=1}^{S\left(T\right)}\frac{1}{2}S_{^{}i^{}}^2$$,where $S\left(T\right)$ is the total number of customers, and  is the service time of customer i.
$$E\left\lbrack W_Q\right\rbrack=\frac{E\left\lbrack R\right\rbrack}{1-\rho}$$
$$E\left\lbrack D\right\rbrack=\frac{E\left\lbrack R\right\rbrack}{1-\rho}+\frac{1}{\mu}$$
$$E\left\lbrack Q\right\rbrack=\lambda\frac{E\left\lbrack R\right\rbrack}{1-\rho}+\rho$$

## M/M/1
For M/M/1 with $\lambda=3$ and $\mu=4$, theoretically, $E\left\lbrack R\right\rbrack=\frac{\rho}{\mu}=0.1875$, $E\left\lbrack Q\right\rbrack=\frac{\rho}{1-\rho}=3$, $E\left\lbrack D\right\rbrack=\frac{1}{\mu-\lambda}=1$. The total number of arrivals is 50000.

200 M/M/1 models with the above parameters are generated. The mean queue size, mean delay, and mean residual time are measured by PK Formula. Their confidence intervals are also calculated.

In [30]:
def general_simulation_mm1(lam, mu, total_arrivals):
    arrival_time = np.zeros(total_arrivals) # the time when the customer arrives
    service_time = np.zeros(total_arrivals) # the service time
    enter_time = np.zeros(total_arrivals)   # the time when the customer is served
    departure_time = np.zeros(total_arrivals)  # the time when the customer leaves the queue

    # initialization of service time and arrival time
    for i in range(total_arrivals):
        service_time[i] = random.expovariate(mu)
    for i in range(1, total_arrivals):
        arrival_time[i] = arrival_time[i - 1] + random.expovariate(lam)

    departure_time[0] = service_time[0]

    # simulation process
    for i in range(1, total_arrivals):
        if departure_time[i - 1] < arrival_time[i]:
            enter_time[i] = arrival_time[i]
        else:
            enter_time[i] = departure_time[i - 1]
        departure_time[i] = enter_time[i] + service_time[i]

    delay = departure_time - arrival_time
    mean_queue_size = np.sum(delay) / departure_time[total_arrivals - 1]
    mean_delay = np.mean(delay)
    mean_residual_time = 0.5 * np.sum(np.square(service_time)) / departure_time[total_arrivals - 1]

    return mean_queue_size, mean_delay, mean_residual_time

In [31]:
lam = 3
mu = 4
total_arrivals = 50000
loop_time = 200
EQ = np.zeros(loop_time)
ED = np.zeros(loop_time)
EQ_PK = np.zeros(loop_time)
ED_PK = np.zeros(loop_time)
ER = np.zeros(loop_time)

for i in range(loop_time):
    EQ[i], ED[i], ER[i] = general_simulation_mm1(lam, mu, total_arrivals)
    ED_PK[i] = ER[i] / (1 - lam / mu) + 1/ mu
    EQ_PK[i] = lam * ED_PK[i]

residual_time_confidence_interval = stats.norm.interval(alpha = 0.95,
                                                        loc = np.mean(ER),
                                                        scale = stats.sem(ER))
queue_confidence_interval = stats.norm.interval(alpha = 0.95,
                                                loc = np.mean(EQ_PK),
                                                scale = stats.sem(EQ_PK))
delay_confidence_interval = stats.norm.interval(alpha = 0.95,
                                                loc = np.mean(ED_PK),
                                                scale = stats.sem(ED_PK))

In [32]:
rho = lam / mu
print("M/M/1 theoretical mean residual time:", rho / mu)
print("M/M/1 mean residual time:", np.mean(ER))
print("Residual time confidence intervals from PK Formula", residual_time_confidence_interval)
print("----------------------------------------------------------------------------------------------")
print("M/M/1 theoretical mean queue size:", rho / (1 - rho))
print("M/M/1 mean queue size from general simulation:", np.mean(EQ))
print("M/M/1 mean queue size from PK Formula:", np.mean(EQ_PK))
print("Queue size confidence intervals from PK Formula", queue_confidence_interval)
print("----------------------------------------------------------------------------------------------")
print("M/M/1 theoretical mean delay:", 1 / (mu - lam))
print("M/M/1 mean delay from general simulation:", np.mean(ED))
print("M/M/1 mean delay from PK Formula:", np.mean(ED_PK))
print("Delay confidence intervals from PK Formula", delay_confidence_interval)

M/M/1 theoretical mean residual time: 0.1875
M/M/1 mean residual time: 0.18746242263119955
Residual time confidence intervals from PK Formula (0.18718524966260047, 0.18773959559979864)
----------------------------------------------------------------------------------------------
M/M/1 theoretical mean queue size: 3.0
M/M/1 mean queue size from general simulation: 2.999135445520892
M/M/1 mean queue size from PK Formula: 2.9995490715743944
Queue size confidence intervals from PK Formula (2.9962229959512054, 3.0028751471975834)
----------------------------------------------------------------------------------------------
M/M/1 theoretical mean delay: 1.0
M/M/1 mean delay from general simulation: 0.9996381300017299
M/M/1 mean delay from PK Formula: 0.9998496905247982
Delay confidence intervals from PK Formula (0.9987409986504019, 1.0009583823991945)


## M/D/1
The generation process of M/D/1 is very similar to M/M/1's. The only difference is the service time initialization. Since the input is the service rate d, the service time should be 1 / d.

For M/D/1 with $\lambda=3$ and $\mu=4$, theoretically, $E\left\lbrack R\right\rbrack=\frac{\rho}{2\mu}=0.09375$, $E\left\lbrack Q\right\rbrack=\frac{\rho}{1-\rho}\times\frac{2-\rho}{2}=1.875$, $E\left\lbrack D\right\rbrack=\frac{1}{\lambda}\times\frac{\rho}{1-\rho}\times\frac{2-\rho}{2}=0.625$. The total number of arrivals is 50000.

200 M/D/1 models with the above parameters are generated. The mean queue size, mean delay, and mean residual time are measured by PK Formula. Their confidence intervals are also calculated.

In [33]:
def general_simulation_md1(lam, d, total_arrivals):
    arrival_time = np.zeros(total_arrivals) # the time when the customer arrives
    service_time = np.zeros(total_arrivals) # the service time
    enter_time = np.zeros(total_arrivals)   # the time when the customer is served
    departure_time = np.zeros(total_arrivals)  # the time when the customer leaves the queue

    # initialization of service time and arrival time
    for i in range(total_arrivals):
        service_time[i] = 1 / d
    for i in range(1, total_arrivals):
        arrival_time[i] = arrival_time[i - 1] + random.expovariate(lam)

    departure_time[0] = service_time[0]

    # simulation process
    for i in range(1, total_arrivals):
        if departure_time[i - 1] < arrival_time[i]:
            enter_time[i] = arrival_time[i]
        else:
            enter_time[i] = departure_time[i - 1]
        departure_time[i] = enter_time[i] + service_time[i]

    delay = departure_time - arrival_time
    mean_queue_size = np.sum(delay) / departure_time[total_arrivals - 1]
    mean_delay = np.mean(delay)
    mean_residual_time = 0.5 * np.sum(np.square(service_time)) / departure_time[total_arrivals - 1]

    return mean_queue_size, mean_delay, mean_residual_time

In [34]:
lam = 3
d = 4
total_arrivals = 50000
loop_time = 200
EQ = np.zeros(loop_time)
ED = np.zeros(loop_time)
EQ_PK = np.zeros(loop_time)
ED_PK = np.zeros(loop_time)
ER = np.zeros(loop_time)

for i in range(loop_time):
    EQ[i], ED[i], ER[i] = general_simulation_md1(lam, d, total_arrivals)
    ED_PK[i] = ER[i] / (1 - lam / mu) + 1/ mu
    EQ_PK[i] = lam * ED_PK[i]

residual_time_confidence_interval = stats.norm.interval(alpha = 0.95,
                                                        loc = np.mean(ER),
                                                        scale = stats.sem(ER))
queue_confidence_interval = stats.norm.interval(alpha = 0.95,
                                                loc = np.mean(EQ_PK),
                                                scale = stats.sem(EQ_PK))
delay_confidence_interval = stats.norm.interval(alpha = 0.95,
                                                loc = np.mean(ED_PK),
                                                scale = stats.sem(ED_PK))

In [35]:
rho = lam / d
print("M/D/1 theoretical mean residual time:", rho / (2 * mu))
print("M/D/1 mean residual time:", np.mean(ER))
print("Residual time confidence intervals from PK Formula", residual_time_confidence_interval)
print("----------------------------------------------------------------------------------------------")
print("M/D/1 theoretical mean queue size:", rho + 0.5 * (rho * rho / (1 - rho)))
print("M/D/1 mean queue size from general simulation:", np.mean(EQ))
print("M/D/1 mean queue size from PK Formula:", np.mean(EQ_PK))
print("Queue size confidence intervals from PK Formula", queue_confidence_interval)
print("----------------------------------------------------------------------------------------------")
print("M/D/1 theoretical mean delay:", 1 / mu + rho / (2 * mu * (1 - rho)))
print("M/D/1 mean delay from general simulation:", np.mean(ED))
print("M/D/1 mean delay from PK Formula:", np.mean(ED_PK))
print("Delay confidence intervals from PK Formula", delay_confidence_interval)

M/D/1 theoretical mean residual time: 0.09375
M/D/1 mean residual time: 0.0937252659492502
Residual time confidence intervals from PK Formula (0.09366583160341198, 0.09378470029508841)
----------------------------------------------------------------------------------------------
M/D/1 theoretical mean queue size: 1.875
M/D/1 mean queue size from general simulation: 1.873730905943499
M/D/1 mean queue size from PK Formula: 1.8747031913910026
Queue size confidence intervals from PK Formula (1.873989979240944, 1.8754164035410612)
----------------------------------------------------------------------------------------------
M/D/1 theoretical mean delay: 0.625
M/D/1 mean delay from general simulation: 0.6247100110675019
M/D/1 mean delay from PK Formula: 0.6249010637970008
Delay confidence intervals from PK Formula (0.624663326413648, 0.6251388011803537)
